In [ ]:
#| default_exp data.postproc

# Postprocessing

> Smoothing, combining etc.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon

## Non-maximum suppression

First the commonly used NMS with bounding boxes, that prioritizes either confidence score (default) or bounding box area.

In [ ]:
#| export
 
# Malisiewicz et al.
def non_max_suppression_fast(boxes, scores, overlap_thresh:float, sort_criterion:str='score'):
    "Possibility to sort boxes by score (default) or area"
    
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
 
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    # sort prediction by scores, 

    # initialize the list of picked indexes    
    pick = []
 
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    if sort_criterion == 'score':
        idxs = np.argsort(scores)
    elif sort_criterion == 'area':
        idxs = np.argsort(area)
    else:
        print('Unknown sort criteria, reverting to "score"')
        idxs = np.argsort(scores)

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
 
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
 
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
 
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
                         np.where(overlap > overlap_thresh)[0])))
 
    # return indices for selected bounding boxes
    return pick
    #return boxes[pick].astype("int")

Non-max suppression can in theory be applied also on polygons, but it hasn't been used in any publications as far as I know.

If `non_max_suppression_poly` is used to eliminate polygons, threshold might need to be smaller than typical value of 0.7 that is used.

In [ ]:
#| export

def poly_IoU(poly_1:Polygon, poly_2:Polygon) -> float:
    "Calculate IoU for two shapely Polygons"
    area_intersection = poly_1.intersection(poly_2).area
    area_union = poly_1.union(poly_2).area
    iou = area_intersection / area_union
    return iou

In [ ]:
#| export

def non_max_suppression_poly(geoms, scores, overlap_thresh:float, sort_criterion:str='score'):
    "Do non-max suppression for shapely Polygons in `geoms`. Can be sorted according to `area` or `score`"
    
    # if there are no geoms, return an empty list
    if len(geoms) == 0:
        return []

    # sort prediction by scores, 

    # initialize the list of picked indexes    
    pick = []
 
    # compute the area of the bounding geoms and sort the bounding
    # geoms by the bottom-right y-coordinate of the bounding box
    area = np.array([geom.area for geom in geoms])
    if sort_criterion == 'score':
        idxs = np.argsort(scores)
    elif sort_criterion == 'area':
        idxs = np.argsort(area)
    else:
        print('Unknown sort criteria, reverting to "score"')
        idxs = np.argsort(scores)

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # compute the ratio of overlap with all other polygons
        overlap = np.array([poly_IoU(geoms[i], geoms[k]) for k in idxs[:last]])
        # delete all indexes from the index list that have
        # overlap larger than overlap_thresh
        idxs = np.delete(idxs, np.concatenate(([last],
                         np.where(overlap > overlap_thresh)[0])))
         
    # return indices for selected bounding geoms
    return pick

Some utils to run above functions to `GeoDataFrames`

In [ ]:
#| export

def do_nms(gdf:gpd.GeoDataFrame, nms_thresh=0.7, crit='score') -> gpd.GeoDataFrame:
    "Perform non-max suppression for bounding boxes using `nms_threshold` to `gdf`"
    gdf = gdf.copy()
    np_bboxes = np.array([b.bounds for b in gdf.geometry])
    scores = gdf.score.values
    idxs = non_max_suppression_fast(np_bboxes, scores, nms_thresh, crit)
    gdf = gdf.iloc[idxs]
    return gdf

In [ ]:
def do_poly_nms(gdf:gpd.GeoDataFrame, nms_thresh=0.1, crit='score') -> gpd.GeoDataFrame:
    "Perform non-max suppression for polygons using `nms_threshold` to `gdf`"
    gdf = gdf.copy()
    scores = gdf.score.values
    idxs = non_max_suppression_poly(gdf.geometry.values, scores, nms_thresh, crit)
    gdf = gdf.iloc[idxs]
    return gdf

In [ ]:
def do_min_rot_rectangle_nms(gdf:gpd.GeoDataFrame, nms_thresh=0.7, crit='score') -> gpd.GeoDataFrame:
    "Perform non-max suppression for rotated bounding boxes using `nms_threshold` to `gdf`"
    gdf = gdf.copy()
    scores = gdf.score.values
    boxes = np.array([g.minimum_rotated_rectangle for g in gdf.geometry.values])
    idxs = non_max_suppression_poly(boxes, scores, nms_thresh, crit)
    gdf = gdf.iloc[idxs]
    return gdf